Basierend auf https://data-dive.com/german-nlp-binary-text-classification-of-reviews-part1/

In [ ]:
import re
import pickle
import sklearn
import pandas as pd
import numpy as np
from bokeh.io import output_notebook

output_notebook()

from hvplot import pandas
from pathlib import Path

pd.options.display.max_columns = 100
pd.options.display.max_rows = 300
pd.options.display.max_colwidth = 100
np.set_printoptions(threshold=2000)

In [ ]:
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier

In [ ]:
data_dir_pp = Path("../../data/party_programs")
df_pp = pd.read_parquet(data_dir_pp / "party_programs.parquet")

data_dir_speeches = Path("../../data/speeches")
df_speeches = pd.read_parquet(data_dir_speeches / "speeches.parquet")

df_sample = pd.concat([df_pp, df_speeches], ignore_index=True).sample(20000, random_state=42)

In [ ]:
from bokeh.models import NumeralTickFormatter

word_freq = pd.Series(" ".join(df_sample["tokenized_text"]).split()).value_counts()
word_freq[1:40].rename("Word frequency of most common words in comments").hvplot.bar(rot=45).opts(
    width=700, height=400, yformatter=NumeralTickFormatter(format="0,0")
)

In [ ]:
vectorizer = TfidfVectorizer(analyzer="word", max_df=0.3, min_df=10, ngram_range=(1, 2), norm="l2")
vectorizer.fit(df_sample["tokenized_text"])

In [ ]:
train, test = train_test_split(df_sample, random_state=1, test_size=0.25, shuffle=True)

X_train = train["tokenized_text"]
Y_train = train["party"]
X_test = test["tokenized_text"]
Y_test = test["party"]
print(X_train.shape)
print(X_test.shape)

In [ ]:
X_train_vec = vectorizer.transform(X_train)
X_test_vec = vectorizer.transform(X_test)
X_train_vec.get_shape()

In [ ]:
classifiers = [
    LogisticRegression(solver="sag", multi_class="multinomial"),
    LinearSVC(multi_class="crammer_singer"),
    KNeighborsClassifier(),
    RandomForestClassifier(),
    DecisionTreeClassifier(),
    ExtraTreeClassifier(),
    GradientBoostingClassifier(),
    MultinomialNB(),
    SGDClassifier(),
    MLPClassifier(
        solver="adam",
        hidden_layer_sizes=(12, 12, 12),
        activation="relu",
        early_stopping=True,
        n_iter_no_change=1,
    ),
]
names = [re.match(r"[^\(]+", name.__str__())[0] for name in classifiers]

In [ ]:
results = {}
for name, clf in zip(names, classifiers):
    print(f"Training classifier: {name}")
    clf.fit(X_train_vec, Y_train)
    prediction = clf.predict(X_test_vec)
    report = sklearn.metrics.classification_report(Y_test, prediction)
    results[name] = report

In [ ]:
for k, v in results.items():
    print(f"Results for {k}:")
    print(f"{v}\n")

## Undersampling

In [ ]:
from imblearn.under_sampling import RandomUnderSampler

In [ ]:
X_train_vec.shape

In [ ]:
# show class distribution
print("Original class distribution:")
print(Y_train.value_counts(normalize=True))

In [ ]:
rus = RandomUnderSampler(random_state=42)
X_train_res, y_train_res = rus.fit_resample(X_train_vec, Y_train)

In [ ]:
X_train_res.shape

In [ ]:
# show class distribution
print("Resampled class distribution:")
print(y_train_res.value_counts(normalize=True))

In [ ]:
results = {}
for name, clf in zip(names, classifiers):
    print(f"Training classifier: {name}")
    clf.fit(X_train_res, y_train_res)
    prediction = clf.predict(X_test_vec)
    report = sklearn.metrics.classification_report(Y_test, prediction)
    results[name] = report

In [ ]:
for k, v in results.items():
    print(f"Results for {k}:")
    print(f"{v}\n")

## Oversampling

In [ ]:
from imblearn.over_sampling import RandomOverSampler

In [ ]:
rus = RandomOverSampler(random_state=42)
X_train_res, y_train_res = rus.fit_resample(X_train_vec, Y_train)

In [ ]:
print(X_train_res.shape)

In [ ]:
# show class distribution
print("Resampled class distribution:")
print(y_train_res.value_counts(normalize=True))

In [ ]:
results = {}
for name, clf in zip(names, classifiers):
    print(f"Training classifier: {name}")
    clf.fit(X_train_res, y_train_res)
    prediction = clf.predict(X_test_vec)
    report = sklearn.metrics.classification_report(Y_test, prediction)
    results[name] = report

In [ ]:
for k, v in results.items():
    print(f"Results for {k}:")
    print(f"{v}\n")

In [ ]:
pipe = Pipeline([("tfidf", TfidfVectorizer()), ("svc", LinearSVC())])

params = {
    "tfidf__ngram_range": [(1, 1), (1, 2), (1, 3)],
    "tfidf__max_df": np.arange(0.3, 0.8, 0.2),
    "tfidf__min_df": np.arange(5, 100, 45),
}
pipe_clf = GridSearchCV(pipe, params, n_jobs=-1, scoring="f1_macro")
pipe_clf.fit(X_train, Y_train)

In [ ]:
print(pipe_clf.best_params_)

In [ ]:
pipe = Pipeline([("tfidf", TfidfVectorizer()), ("svc", LinearSVC())])

params = {
    "tfidf__ngram_range": [(1, 3)],
    "tfidf__max_df": [0.5],
    "tfidf__min_df": [5],
    "svc__C": np.arange(0.2, 1, 0.15),
}
pipe_svc_clf = GridSearchCV(pipe, params, n_jobs=-1, scoring="f1_macro")
pipe_svc_clf.fit(X_train, Y_train)

In [ ]:
best_params = pipe_svc_clf.best_params_
print(best_params)

In [ ]:
pipe.set_params(**best_params).fit(X_train, Y_train)
pipe_pred = pipe.predict(X_test)
report = sklearn.metrics.classification_report(Y_test, pipe_pred)
print(report)

In [ ]:
conf_score = pipe.decision_function(X_test)

In [ ]:
test_df = test.copy()
test_df["pred_party"] = pipe_pred
test_df["pred_score"] = conf_score.max(axis=1)
test_df[["party", "pred_party", "clean_text", "pred_score"]][(test_df["party"] != test_df["pred_party"])].sort_values(
    by="pred_score", ascending=False
).head(10)